In [ ]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# math
import numpy as np
import torch
import pytorch3d
import pytorch3d.transforms
# physics
# plots
import matplotlib as mpl
from matplotlib import pyplot as plt
from src.plot import set_fig_size, set_mpl_format
# config
from src.config import cfg
# dataset
from src.dataset import load_train_data
from src.data.visualize import print_example

# setup
set_mpl_format()

### Config

In [ ]:
# fill in your directory set up here
config_fp = os.path.join(project_dir, f'cfg/templates/comp-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.DATA.path = os.path.join(project_dir, 'data/sets/')
print(cfg)

### Data

In [ ]:
train_ds, ds_info = load_train_data(cfg)
example = next(iter(train_ds))

In [ ]:
vid_id = example['metadata']['video_name']
obj_count = example['metadata']['num_instances']
print(f'Video: {vid_id}, Num objects: {obj_count}')

### Scene